In [10]:
!pip install datasets transformers tokenizers accelerate torch torchvision torchaudio evaluate scikit-learn
!pip install huggingface-hub



/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


^C
ERROR: Operation cancelled by user


In [11]:
from huggingface_hub import login
login()

In [1]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("SKNahin/bengali-transliteration-data")

# Split the dataset into training and validation sets (e.g., 80/20)
train_data = dataset['train'].train_test_split(test_size=0.2)
train_data, validation_data = train_data['train'], train_data['test']


README.md:   0%|          | 0.00/300 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/333k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5006 [00:00<?, ? examples/s]

In [13]:
print(train_data[0])  
print(train_data.column_names)  


{'bn': 'এটা কোনো পোস্ট হলো মিয়া আবাল', 'rm': 'eta kono post holo mia abal'}
['bn', 'rm']


In [14]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("t5-small")


def preprocess_function(examples):
    inputs = examples["rm"]  
    targets = examples["bn"] 
 
    model_inputs = tokenizer(inputs, max_length=64, truncation=True, padding="max_length")
    
  
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=64, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


train_dataset = train_data.map(preprocess_function, batched=True)
validation_dataset = validation_data.map(preprocess_function, batched=True)


Map:   0%|          | 0/4004 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1002 [00:00<?, ? examples/s]

In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/working/results",  
    evaluation_strategy="epoch", 
    learning_rate=2e-5,  
    per_device_train_batch_size=8, 
    per_device_eval_batch_size=8,  
    weight_decay=0.01,  
    save_total_limit=3,  
    num_train_epochs=1, 
    predict_with_generate=True,  
    logging_dir="/kaggle/working/logs",  
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
model_name = "t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)


trainer.train()

<IPython.core.display.Javascript object>

In [ ]:
model_save_path = "/kaggle/working/banglish_to_bangla_model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

In [ ]:
def translate_banglish_to_bangla(input_text):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=64, truncation=True)
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


test_text = "Vai…link ai to dhoka jaccena…karo kace link thakle daw"
print("Translated Text:", translate_banglish_to_bangla(test_text))